In [1]:
import pandapower as pp
from faraday import DATA_DIR
from copy import deepcopy
import json

In [2]:
test_dir = DATA_DIR / "test_networks" / "cigre_mv"

In [3]:
network = pp.networks.create_cigre_network_mv(with_der="pv_wind")
network.load["curtailable"] = False

In [4]:
pp.runpp(network)

In [5]:
pp.plotting.plotly.pf_res_plotly(network, auto_open=False)

In [6]:
org_network = deepcopy(network)

In [7]:
network.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,curtailable
0,Load R1,1,14.99400,3.044662,0.0,0.0,15.300,1.0,True,wye,False
1,Load R3,3,0.27645,0.069285,0.0,0.0,0.285,1.0,True,wye,False
2,Load R4,4,0.43165,0.108182,0.0,0.0,0.445,1.0,True,wye,False
3,Load R5,5,0.72750,0.182329,0.0,0.0,0.750,1.0,True,wye,False
4,Load R6,6,0.54805,0.137354,0.0,0.0,0.565,1.0,True,wye,False
5,Load R8,8,0.58685,0.147078,0.0,0.0,0.605,1.0,True,wye,False
6,Load R10,10,0.47530,0.119121,0.0,0.0,0.490,1.0,True,wye,False
7,Load R11,11,0.32980,0.082656,0.0,0.0,0.340,1.0,True,wye,False
8,Load R12,12,14.99400,3.044662,0.0,0.0,15.300,1.0,True,wye,False
9,Load R14,14,0.20855,0.052268,0.0,0.0,0.215,1.0,True,wye,False


In [8]:
network.switch

,bus,element,et,type,closed,name,z_ohm,in_ka
0,6,12,l,LBS,True,None,0.0,NaN
1,7,12,l,LBS,False,S2,0.0,NaN
2,4,13,l,LBS,False,S3,0.0,NaN
3,11,13,l,LBS,True,None,0.0,NaN
4,8,14,l,LBS,False,S1,0.0,NaN
5,14,14,l,LBS,True,None,0.0,NaN
6,0,0,t,CB,True,None,0.0,NaN
7,0,1,t,CB,True,None,0.0,NaN


# Test Cases

## 1. Add large load on Bus 7

In [9]:
network.load.loc[network.load.bus == 7, "p_mw"] = 5
network.load.loc[network.load.bus == 7, "curtailable"] = True

test_network1 = deepcopy(network)

In [10]:
pp.runpp(network)
pp.plotting.plotly.pf_res_plotly(network, auto_open=False)

In [11]:
network.res_bus[(network.res_bus.vm_pu > 1.05) | (network.res_bus.vm_pu < 0.95)]

,vm_pu,va_degree,p_mw,q_mvar
2,0.943385,-39.716673,0.00000,0.000000
3,0.880536,-43.338927,0.48170,0.208882
4,0.878631,-43.448485,0.41165,0.108182
5,0.877325,-43.523953,0.69750,0.182329
6,0.875797,-43.612803,0.51805,0.137354
7,0.859360,-44.993286,3.50000,0.047410
8,0.868088,-44.195429,0.55685,0.147078
9,0.867075,-44.237497,0.54375,0.355578
10,0.865796,-44.307391,0.50330,0.161264
11,0.865593,-44.319516,0.31980,0.082656


In [12]:
network.res_line[network.res_line.loading_percent > 100]

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,8.346079,2.360651,-8.071719,-2.018376,0.274360,0.342276,0.254215,0.254599,0.254599,0.984926,-37.638172,0.943385,-39.716673,175.585433
1,8.071719,2.018376,-7.640152,-1.471522,0.431567,0.546854,0.254599,0.255078,0.255078,0.943385,-39.716673,0.880536,-43.338927,175.916092
9,5.526335,0.870890,-5.460571,-0.795783,0.065764,0.075106,0.183411,0.183505,0.183505,0.880536,-43.338927,0.868088,-44.195429,126.555021


In [13]:
network.switch.loc[:, "closed"] = True
pp.runpp(network)
pp.plotting.plotly.pf_res_plotly(network, auto_open=False)

In [14]:
network.res_bus[(network.res_bus.vm_pu > 1.05) | (network.res_bus.vm_pu < 0.95)]

,vm_pu,va_degree,p_mw,q_mvar
3,0.938544,-38.704808,0.48170,0.208882
4,0.935586,-38.863372,0.41165,0.108182
5,0.933724,-38.989150,0.69750,0.182329
6,0.930545,-39.231103,0.51805,0.137354
7,0.930275,-39.256661,3.50000,0.047410
8,0.936405,-38.749591,0.55685,0.147078
9,0.935717,-38.786975,0.54375,0.355578
10,0.935134,-38.849858,0.50330,0.161264
11,0.935203,-38.861465,0.31980,0.082656
14,0.946857,-38.309632,0.54005,0.257713


In [15]:
network.res_line[network.res_line.loading_percent > 100]

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent


In [16]:
pp.create_storage(network, 7, p_mw=-1, max_e_mwh=1)
pp.create_storage(network, 7, p_mw=-1, max_e_mwh=1)
pp.create_storage(network, 7, p_mw=-1, max_e_mwh=1)

pp.runpp(network)
pp.plotting.plotly.pf_res_plotly(network, auto_open=False)

In [17]:
network.res_bus[(network.res_bus.vm_pu > 1.05) | (network.res_bus.vm_pu < 0.95)]

,vm_pu,va_degree,p_mw,q_mvar


In [18]:
network.res_line[network.res_line.loading_percent > 100]

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent


In [19]:
test_case_dir = test_dir / "cigre_mv_1_large_load"
test_case_dir.mkdir(parents=True, exist_ok=True)

pp.to_json(test_network1, test_case_dir / "network.json")

with open(test_case_dir / "metadata.json", "+w") as f:
    json.dump(
        {
            "base_network": "cigre_mv",
            "severity": "serve",
            "test_case": "1_large_loads",
            "solutions": [
                {
                    "switch": [
                        {"name": "S1", "closed": True},
                        {"name": "S2", "closed": True},
                        {"name": "S3", "closed": True},
                    ],
                    "battery": [
                        {"bus": 7, "p_mw": 1},
                        {"bus": 7, "p_mw": 1},
                        {"bus": 7, "p_mw": 1},
                    ],
                }
            ],
        },
        f,
        indent=2,
    )

## Test case 2: Two Small loads

In [20]:
network = deepcopy(org_network)
network.load.loc[network.load.bus == 6, "p_mw"] += 2
network.load.loc[network.load.bus == 11, "p_mw"] += 0.7
test_network2 = deepcopy(network)
pp.runpp(network)
pp.plotting.plotly.pf_res_plotly(network, auto_open=False)

In [21]:
network.switch.loc[:, "closed"] = True
pp.runpp(network)
pp.plotting.plotly.pf_res_plotly(network, auto_open=False)

In [22]:
network.res_bus[(network.res_bus.vm_pu > 1.05) | (network.res_bus.vm_pu < 0.95)]

,vm_pu,va_degree,p_mw,q_mvar


In [23]:
network.res_line[network.res_line.loading_percent > 100]

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent


In [24]:
test_case_dir = test_dir / "cigre_mv_2_small_loads"
test_case_dir.mkdir(parents=True, exist_ok=True)

pp.to_json(test_network2, test_case_dir / "network.json")

with open(test_case_dir / "metadata.json", "+w") as f:
    json.dump(
        {
            "base_network": "cigre_mv",
            "severity": "medium",
            "test_case": "2_small_loads",
            "solutions": [
                {
                    "switch": [
                        {"name": "S1", "closed": True},
                        {"name": "S2", "closed": True},
                        {"name": "S3", "closed": True},
                    ],
                }
            ],
        },
        f,
        indent=2,
    )

## Test Case 3: Line Drop

In [25]:
network = deepcopy(org_network)
pp.drop_lines(network, [9])
test_network3 = deepcopy(network)
pp.runpp(network)
pp.plotting.plotly.pf_res_plotly(network, auto_open=False)

In [26]:
network.switch.loc[network.switch.name == "S1", "closed"] = True
pp.runpp(network)
pp.plotting.plotly.pf_res_plotly(network, auto_open=False)

In [27]:
network.res_bus[(network.res_bus.vm_pu > 1.05) | (network.res_bus.vm_pu < 0.95)]

,vm_pu,va_degree,p_mw,q_mvar


In [28]:
network.res_line[network.res_line.loading_percent > 100]

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent


In [29]:
test_case_dir = test_dir / "cigre_mv_drop_1_line"
test_case_dir.mkdir(parents=True, exist_ok=True)

pp.to_json(test_network3, test_case_dir / "network.json")

with open(test_case_dir / "metadata.json", "+w") as f:
    json.dump(
        {
            "base_network": "cigre_mv",
            "severity": "severe",
            "test_case": "drop_1_line",
            "solutions": [
                {
                    "switch": [
                        {"name": "S1", "closed": True},
                    ]
                }
            ],
        },
        f,
        indent=2,
    )